# Chart Library Notebook

This notebook will begin the basis for production level chart library using the agmktdb.

In [1]:
import os

# Data science
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Database
from sqlalchemy import create_engine
import psycopg2
import agmkt_pkg.nass as nass
import agmkt_pkg.env_vars as env_vars


print('Libraries imported!!')

Libraries imported!!


# Create SELECT statement to pull raw data

In [2]:
# create sql connection
user = env_vars.mktdb_user()
password = env_vars.mktdb_passwd()
server = env_vars.mktdb_server()


connection_string = f'postgresql://{user}:{password}@{server}/mktdb'
eng = create_engine(connection_string)
conn = eng.connect()

In [3]:
# select_stmt = '''
# SELECT "SHORT_DESC", "YEAR", "VALUE"
# FROM nass_crops_fmt
# WHERE 
#     "SOURCE_DESC" in ('SURVEY')
#     AND "COMMODITY_DESC" in ('CORN')
#     AND "PRODN_PRACTICE_DESC" in ('ALL PRODUCTION PRACTICES')
#     AND "UTIL_PRACTICE_DESC" in ('GRAIN')
#     AND "STATISTICCAT_DESC" in ('YIELD')
#     AND "AGG_LEVEL_DESC" in ('NATIONAL')
#     AND "REFERENCE_PERIOD_DESC" in ('YEAR')

# '''

# df = pd.read_sql(select_stmt, con = conn)

In [4]:
stmt_dict = {'COMMODITY_DESC': 'OATS', 'PRODN_PRACTICE_DESC':'ALL PRODUCTION PRACTICES', 
             #'UTIL_PRACTICE_DESC':'GRAIN', 
             'STATISTICCAT_DESC':'PRODUCTION', 'UNIT_DESC':'BU', 'AGG_LEVEL_DESC':'NATIONAL', 'REFERENCE_PERIOD_DESC':'YEAR'}
# filter_string = ''

# for key, value in stmt_dict.items():
#         filter_string += f""" AND "{key}" IN ('{value}')"""

# print(filter_string)

df = nass.crops_time_series_chart(stmt_dict=stmt_dict, db_connect=conn)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Manimpulate data for data visualization

In [ ]:
# df['date'] = df['YEAR'].astype('str') + '-09-01'
# df['date'] = pd.to_datetime(df['date'])

# Create Visualziation and save image to folder

In [ ]:
df_viz = df[df['date'].dt.year >= df['date'].dt.year.max() - 50]

str_commodity = stmt_dict['COMMODITY_DESC'].title()
str_statistic = stmt_dict['STATISTICCAT_DESC'].title()
str_unit = stmt_dict['UNIT_DESC'].title()

plt.subplot()
sns.lineplot(data=df_viz, x='date', y='VALUE')
sns.scatterplot(data=df_viz, x='date', y='VALUE')
plt.title(f'U.S. {str_commodity} {str_statistic}')
plt.ylabel(f'{str_unit}')
plt.xlabel('Crop Year')
plt.show()